In [29]:
import sqlite3
import decimal
conn = sqlite3.connect('quake_mmi_data.db')
c = conn.cursor()


# # drop table if working
# c.execute('''DROP TABLE quake_mmi_test''')

# execute query to create table 
c.execute('''CREATE TABLE quake_mmi_test
(	
	ID INT NOT NULL,
  	publicID VARCHAR (10) NOT NULL,
 	date date (50) NOT NULL,
	mmi INT NOT NULL,
	magnitude FLOAT NOT NULL,
	locality VARCHAR (250) NOT NULL,
	longitude FLOAT NOT NULL,
	latitude FLOAT NOT NULL,
	time timestamp NOT NULL,
	depth FLOAT NOT NULL,
	PRIMARY KEY (ID)
)''')


In [30]:
# Load this CSV data into a sqlite table

import pandas as pd
# load the data into a Pandas DataFrame
# quake_mmi = pd.read_csv('quake_mmi_data.csv')

quake_mmi['date'] = pd.to_datetime(quake_mmi['date'])
quake_mmi['time'] = pd.to_datetime(quake_mmi['time'])
quake_mmi.dtypes

# write the data to a sqlite table
quake_mmi.to_sql('quake_mmi_test', conn, if_exists='replace', index = True)


294

In [31]:
quake_mmi.dtypes

publicID                  object
date              datetime64[ns]
mmi                        int64
magnitude                float64
locality                  object
longitude                float64
latitude                 float64
time         datetime64[ns, UTC]
depth                    float64
dtype: object

In [54]:
import sqlite3
import decimal
conn = sqlite3.connect('quake_mmi_data.db')
c = conn.cursor()

c.execute('''SELECT * from quake_mmi_test''').fetchall() # [(1, 'pokerkid'), (2, 'crazyken')]
# # c.execute('''PRAGMA table_info(quake_mmi_test)''').fetchall()

[(0,
  '2023p452421',
  '2023-06-17 00:00:00',
  -1,
  1.780997494,
  '40 km north of Kaikoura',
  173.735199,
  -42.02975845,
  '2023-06-17 09:30:18.746000+00:00',
  22.44568825),
 (1,
  '2023p452386',
  '2023-06-17 00:00:00',
  3,
  3.021375756,
  '20 km east of Palmerston North',
  175.8822479,
  -40.40055466,
  '2023-06-17 09:11:40.087000+00:00',
  26.99663353),
 (2,
  '2023p452375',
  '2023-06-17 00:00:00',
  3,
  2.93598542,
  '20 km east of Palmerston North',
  175.8869019,
  -40.39117813,
  '2023-06-17 09:05:50.271000+00:00',
  24.72915649),
 (3,
  '2023p452356',
  '2023-06-17 00:00:00',
  -1,
  0.386625535,
  '35 km east of Taihape',
  176.2091217,
  -39.65148926,
  '2023-06-17 08:55:28.053000+00:00',
  24.88100243),
 (4,
  '2023p452338',
  '2023-06-17 00:00:00',
  -1,
  2.143574257,
  '45 km east of Stratford',
  174.8218079,
  -39.22940063,
  '2023-06-17 08:45:50.111000+00:00',
  28.45860291),
 (5,
  '2023p452303',
  '2023-06-17 00:00:00',
  3,
  2.085746448,
  '5 km south-e

In [39]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func


engine = create_engine("sqlite:///quake_mmi_data.db")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(autoload_with=engine)

# Save reference to the table
quake_fetch = Base.classes.quake_mmi_test

 

AttributeError: quake_mmi_test

In [35]:
# Check and test results

session = Session(engine)
results = session.query(quake_fetch.publicID).all()

session.close()
print(results)


[('2022p465796',), ('2022p529447',), ('2022p557940',), ('2022p564358',), ('2022p582126',), ('2022p674002',), ('2022p680474',), ('2022p684821',), ('2022p714540',), ('2022p725774',), ('2022p757921',), ('2022p816171',), ('2022p844361',), ('2022p845259',), ('2022p857336',), ('2022p901216',), ('2022p902801',), ('2022p977695',), ('2023p007281',), ('2023p007922',), ('2023p087955',), ('2023p117270',), ('2023p122368',), ('2023p140223',), ('2023p154834',), ('2023p157593',), ('2023p159349',), ('2023p168329',), ('2023p170633',), ('2023p175349',), ('2023p178504',), ('2023p184644',), ('2023p187624',), ('2023p187628',), ('2023p189923',), ('2023p192961',), ('2023p200114',), ('2023p202484',), ('2023p202703',), ('2023p204450',), ('2023p204478',), ('2023p204951',), ('2023p205057',), ('2023p205252',), ('2023p205256',), ('2023p205484',), ('2023p205534',), ('2023p207850',), ('2023p212590',), ('2023p216781',), ('2023p223101',), ('2023p226064',), ('2023p228763',), ('2023p229809',), ('2023p236682',), ('2023p23

In [40]:
# import necessary libraries
# from models import create_classes
import os
import decimal
import numpy as np
import sqlalchemy
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, or_
from sqlalchemy.ext.automap import automap_base

from flask import Flask, jsonify, request, render_template, redirect

import warnings
from sqlalchemy.exc import SAWarning
warnings.filterwarnings('ignore',
 r"^Dialect sqlite\+pysqlite does \*not\* support Decimal objects natively\, "
 "and SQLAlchemy must convert from floating point - rounding errors and other "
 "issues may occur\. Please consider storing Decimal numbers as strings or "
 "integers on this platform for lossless storage\.$",
 SAWarning, r'^sqlalchemy\.sql\.type_api$')

import sqlite3
import pickle

engine = create_engine("sqlite:///quake_mmi_data.db")
Base = automap_base()

# reflect the tables
Base.prepare(autoload_with=engine, reflect=True)

# Save reference to the table
print(Base.classes.keys())

quake = Base.classes.quake_mmi_test

# session = Session(engine)

# type(session)

# results = session.query(quake.publicID, quake.date, quake.mmi, quake.magnitude, quake.longitude, quake.latitude, quake.time, quake.depth).all()

# type(results)

# session.close()
    # Convert list of tuples into normal list

# earthquake = list(np.ravel(results))

['quake_mmi']


AttributeError: quake_mmi_test